In [50]:
using Test

function get_verbose()
    verbose::String = "yes"
    return verbose
end

function count_information(vector)
    cases_on_near_side::Int                     = trunc(Int, length(vector)/2)
    cardinal_number_of_max_mersenne_set::Int    = trunc(Int, log2(length(vector)/2))
    max_mersennes::Array{Int,1}                 = [trunc(Int,2^i)-1 for i in 1:cardinal_number_of_max_mersenne_set]
    number_of_firsts_for_pairs::Int             = cardinal_number_of_max_mersenne_set - 1
    cover_on_near_side                          = 0
    mersenne_bands_needed                       = 0
    
    verbose = get_verbose()
    
    for i in 1:cardinal_number_of_max_mersenne_set
        if cover_on_near_side + max_mersennes[i] <= cases_on_near_side
            cover_on_near_side    = cover_on_near_side    + max_mersennes[i]
            mersenne_bands_needed = mersenne_bands_needed + 1
        end
    end
    
    band_indices    = [[0,1]]
    for numbered_mersenne in enumerate(max_mersennes)
        number                  = numbered_mersenne[1]
        
        if 1 < number <= mersenne_bands_needed
            
            previous_last           = band_indices[lastindex(band_indices)][2] # or using the last constructor, could time it
            this_first::Int         = previous_last + 1
            
            value                   = numbered_mersenne[2]
            println("\nvalue of mersenne:\t", value)
            
            this_last::Int          = this_first    + value
            this_band::Array{Int,1} = [this_first, this_last]
            
            push!(band_indices, this_band)
        end
    end
    
    println("\nband indices:\t", band_indices)
    
    maximum_information     = 0
    for m in 1:mersenne_bands_needed
        maximum_information = maximum_information + 1-(1/(2^m))
    end
    
    actual_information      = 0
    alpha                   = vector[1]
    for this_numbered_band_index_pair in enumerate(band_indices)
        order      = this_numbered_band_index_pair[1] 
        first_last = this_numbered_band_index_pair[2]
        first      = first_last[1]
        last       = first_last[2]
        println("this is the band:\t", [first, last])
        if last == 1
            println("base case first comparator, it equals itself and is not counted")
            #this_slice = [vector[last]]
            #actual_information = actual_information + 1-(1/(2^order))
            #println("this is the slice:", vector[last], "\nthis is the info:\t", 1-(1/(2^order)))
        else
            this_slice = vector[first:last-1]
            println("this is the slice:\t", this_slice)
            if this_slice[1] == alpha
                println("seems the start is like alpha")
                if all(y->y==this_slice[1],this_slice)
                    println("the whole slice matches the first and alpha")
                    actual_information = actual_information + 1-(1/(2^order))
                    println("this is the slice:\t", this_slice, "\nthis is the added info:\t", 1-(1/(2^order)))
                end
            end
        end
    end
    information_count = 0
    if actual_information > 0
        information_count = actual_information/maximum_information
    end
    
    if verbose == "yes"
        println("length of vector is: \t\t\t", length(vector), " so half is:\t", cases_on_near_side)
        println("generous max layers log2 vector:\t", cardinal_number_of_max_mersenne_set)
        println("mersenne bands needed:\t\t\t", mersenne_bands_needed)
        println("cardinal number of max mersenne set:\t", cardinal_number_of_max_mersenne_set)
        println("cases on near side:\t\t\t", cases_on_near_side)
        println("cover on near side:\t\t\t", cover_on_near_side)
        println("band indices:\t\t", band_indices)
        println("actual information:\t", actual_information)
        println("maximum information:\t",maximum_information)
        println("information count:\t", information_count)
    end
    
    return information_count
end

function test_count_information()
    references = [  [[1,2,3,4],                                             0.0],
                    [[1,2,3,4,5,6,7,8],                                     0.0],
                    [[1,2,3,4,5,6,7,8,9,10,11,12,13,14],                    0.0],
                    [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],  0.0],
                    [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,
                     21,22,23,24,25,26,27,28,29,30,31,32],                  0.0],

                    [[1,1,1,1,2,2,2,2],                                     1.0], 
                    [[1,1,1,2,1,2,2,2],                                     0.0], 
                    [[1,1,1,1,1,2,2,2,2,2],                                 1.0],
                    [[1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2],         0.46153846153846156],
                    [[1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2],     1.0],
                    [[1,1,1,1,1,1,1,1,1,1,1,2,1,2,2,2,2,2,2,2,2,2,2,2],     0.46153846153846156],
        
                    [[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                      2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2],     1.0],
                    [[1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                      2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2],     0.5384615384615384],
        
                    [[1,2,1,2,1,1,1,1,1,1,1,1,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,
                      2,1,2,1,2,2,2,2,2,2,2,2,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2], 
                                                                            0.0]
    ]

    for ref in references
        vector = ref[1]
        print("\n\nordered cases in space from origin:\t", vector)
        information_ref   = ref[2]
        print("\nreference information ratio:\t\t\t", information_ref)
        
        information_count = count_information(vector)
        print("\nderived information ratio:\t\t\t", information_count)
        
        @test information_count==information_ref
    end
end

test_count_information()




ordered cases in space from origin:	[1, 2, 3, 4]
reference information ratio:			0.0
band indices:	[[0, 1]]
this is the band:	[0, 1]
base case first comparator, it equals itself and is not counted
length of vector is: 			4 so half is:	2
generous max layers log2 vector:	1
mersenne bands needed:			1
cardinal number of max mersenne set:	1
cases on near side:			2
cover on near side:			1
band indices:		[[0, 1]]
actual information:	0
maximum information:	0.5
information count:	0

derived information ratio:			0

ordered cases in space from origin:	[1, 2, 3, 4, 5, 6, 7, 8]
reference information ratio:			0.0
value of mersenne:	3

band indices:	[[0, 1], [2, 5]]
this is the band:	[0, 1]
base case first comparator, it equals itself and is not counted
this is the band:	[2, 5]
this is the slice:	[2, 3, 4]
length of vector is: 			8 so half is:	4
generous max layers log2 vector:	2
mersenne bands needed:			2
cardinal number of max mersenne set:	2
cases on near side:			4
cover on near side:			4
band in

LoadError: [91mThere was an error during testing[39m